In [8]:
import pandas as pd 
import numpy as np
import requests

import glob
import json

from config import API_KEY

In [22]:
path = 'gastos-por-mes' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename,
                     index_col=None,
                     header=0, 
                     sep=';',
                     parse_dates=['Mês Ano'])
    li.append(df)

gastos_2020 = pd.concat(li, axis=0, ignore_index=True)

In [23]:
gastos_2020.head()

,Mês Ano,Órgão Superior,Órgão/Entidade Vinculada,Unidade Gestora,Área de atuação (Função),Subfunção,Programa Orçamentário,Ação Orçamentária,Programa de Governo,Autor Emenda,Plano Orçamentário,Grupo de Despesa,Elemento de Despesa,Modalidade de Despesa,Valor Empenhado,Valor Liquidado,Valor Pago,Valor Restos a Pagar Pagos,Unnamed: 18
0,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,20TP - ATIVOS CIVIS DA UNIAO,00 - NAO ATRIBUIDO,0000,RO00 - ATIVOS CIVIS DA UNIAO - REGRA DE OURO,1 - Pessoal e Encargos Sociais,92 - Despesas de Exercícios Anteriores,90 - Reserva de Contingência,"83.519,78","83.519,78","0,00","0,00",NaN
1,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,"5013 - EDUCACAO SUPERIOR - GRADUACAO, POS-GRAD...",20RK - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,00 - NAO ATRIBUIDO,0000,0000 - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,4 - Investimentos,51 - Obras e Instalações,90 - Reserva de Contingência,"0,00","0,00","3.420,00","0,00",NaN
2,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,571 - Desenvolvimento científico,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,212H - MANUTENCAO DE CONTRATO DE GESTAO COM OR...,00 - NAO ATRIBUIDO,0000,0009 - CENTRO BRASILEIRO DE PESQUISA EM AVALIA...,3 - Outras Despesas Correntes,39 - Outros Serviços de Terceiros - Pessoa Jur...,90 - Reserva de Contingência,"69.517,51","69.517,51","69.517,51","0,00",NaN
3,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,331 - Proteção e benefícios ao trabalhador,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,212B - BENEFICIOS OBRIGATORIOS AOS SERVIDORES ...,00 - NAO ATRIBUIDO,0000,0005 - AUXILIO-ALIMENTACAO DE CIVIS,3 - Outras Despesas Correntes,04 - Contratação por Tempo Determinado,90 - Reserva de Contingência,"83.605,64","80.816,14","62.340,04","0,00",NaN
4,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,"5013 - EDUCACAO SUPERIOR - GRADUACAO, POS-GRAD...",4002 - ASSISTENCIA AO ESTUDANTE DE ENSINO SUPE...,00 - NAO ATRIBUIDO,0000,CV19 - CORONAVIRUS (COVID-19),3 - Outras Despesas Correntes,18 - Auxílio Financeiro a Estudantes,90 - Reserva de Contingência,"4.882.196,00","4.519.800,00","4.519.800,00","0,00",NaN


In [24]:
def formatar_input(df):
    df_copy = df.rename(columns={'Mês Ano':'mes_ano',
                                 'Órgão Superior':'orgao_superior',
                                 'Órgão/Entidade Vinculada':'orgao_ent_vinculada',
                                 'Unidade Gestora':'unid_gestora',
                                 'Área de atuação (Função)':'area_atuacao',
                                 'Subfunção':'subfuncao',
                                 'Programa Orçamentário':'programa_orcamentario',
                                 'Ação Orçamentária':'acao_orcamentaria',
                                 'Programa de Governo':'programa_de_governo',
                                 'Autor Emenda':'autor_emenda',
                                 'Plano Orçamentário':'plano_orcamentario',
                                 'Grupo de Despesa':'grupo_de_despesa',
                                 'Elemento de Despesa':'elemento_de_despesa',
                                 'Modalidade de Despesa':'modalidade_de_despesa',
                                 'Valor Empenhado':'valor_empenhado',
                                 'Valor Liquidado':'valor_liquidado',
                                 'Valor Pago':'valor_pago',
                                 'Valor Restos a Pagar Pagos':'restos_a_pagar_pagos'})
    
    df_copy = df_copy.drop(['Unnamed: 18'], axis=1)
    
    df_copy['valor_empenhado'] = [x.replace('.', '') for x in df_copy['valor_empenhado']]
    df_copy['valor_empenhado'] = [x.replace(' ', '') for x in df_copy['valor_empenhado']]
    df_copy['valor_empenhado'] = [x.replace(',', '.') for x in df_copy['valor_empenhado']]
    df_copy['valor_empenhado'] = df_copy['valor_empenhado'].astype(float)
    
    df_copy['valor_liquidado'] = [x.replace('.', '') for x in df_copy['valor_liquidado']]
    df_copy['valor_liquidado'] = [x.replace(' ', '') for x in df_copy['valor_liquidado']]
    df_copy['valor_liquidado'] = [x.replace(',', '.') for x in df_copy['valor_liquidado']]
    df_copy['valor_liquidado'] = df_copy['valor_liquidado'].astype(float)
    
    df_copy['valor_pago'] = [x.replace('.', '') for x in df_copy['valor_pago']]
    df_copy['valor_pago'] = [x.replace(' ', '') for x in df_copy['valor_pago']]
    df_copy['valor_pago'] = [x.replace(',', '.') for x in df_copy['valor_pago']]
    df_copy['valor_pago'] = df_copy['valor_pago'].astype(float)
        
    df_copy['restos_a_pagar_pagos'] = [x.replace('.', '') for x in df_copy['restos_a_pagar_pagos']]
    df_copy['restos_a_pagar_pagos'] = [x.replace(' ', '') for x in df_copy['restos_a_pagar_pagos']]
    df_copy['restos_a_pagar_pagos'] = [x.replace(',', '.') for x in df_copy['restos_a_pagar_pagos']]
    df_copy['restos_a_pagar_pagos'] = df_copy['restos_a_pagar_pagos'].astype(float)
    
    return df_copy

In [25]:
gastos_2020 = formatar_input(gastos_2020)
gastos_2020.head()

,mes_ano,orgao_superior,orgao_ent_vinculada,unid_gestora,area_atuacao,subfuncao,programa_orcamentario,acao_orcamentaria,programa_de_governo,autor_emenda,plano_orcamentario,grupo_de_despesa,elemento_de_despesa,modalidade_de_despesa,valor_empenhado,valor_liquidado,valor_pago,restos_a_pagar_pagos
0,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,20TP - ATIVOS CIVIS DA UNIAO,00 - NAO ATRIBUIDO,0000,RO00 - ATIVOS CIVIS DA UNIAO - REGRA DE OURO,1 - Pessoal e Encargos Sociais,92 - Despesas de Exercícios Anteriores,90 - Reserva de Contingência,83519.78,83519.78,0.00,0.0
1,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,"5013 - EDUCACAO SUPERIOR - GRADUACAO, POS-GRAD...",20RK - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,00 - NAO ATRIBUIDO,0000,0000 - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,4 - Investimentos,51 - Obras e Instalações,90 - Reserva de Contingência,0.00,0.00,3420.00,0.0
2,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,571 - Desenvolvimento científico,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,212H - MANUTENCAO DE CONTRATO DE GESTAO COM OR...,00 - NAO ATRIBUIDO,0000,0009 - CENTRO BRASILEIRO DE PESQUISA EM AVALIA...,3 - Outras Despesas Correntes,39 - Outros Serviços de Terceiros - Pessoa Jur...,90 - Reserva de Contingência,69517.51,69517.51,69517.51,0.0
3,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,331 - Proteção e benefícios ao trabalhador,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,212B - BENEFICIOS OBRIGATORIOS AOS SERVIDORES ...,00 - NAO ATRIBUIDO,0000,0005 - AUXILIO-ALIMENTACAO DE CIVIS,3 - Outras Despesas Correntes,04 - Contratação por Tempo Determinado,90 - Reserva de Contingência,83605.64,80816.14,62340.04,0.0
4,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,"5013 - EDUCACAO SUPERIOR - GRADUACAO, POS-GRAD...",4002 - ASSISTENCIA AO ESTUDANTE DE ENSINO SUPE...,00 - NAO ATRIBUIDO,0000,CV19 - CORONAVIRUS (COVID-19),3 - Outras Despesas Correntes,18 - Auxílio Financeiro a Estudantes,90 - Reserva de Contingência,4882196.00,4519800.00,4519800.00,0.0


In [28]:
gastos_por_elemento = gastos_2020.groupby("elemento_de_despesa")["valor_pago"].sum().sort_values(ascending=False)
gastos_por_elemento

elemento_de_despesa
11 - Vencimentos e Vantagens Fixas - Pessoal Civil                           7.023207e+08
01 - Aposentadorias do RPPS, Reserva Remunerada e Reformas dos Militares     3.067093e+08
13 - Obrigações Patronais                                                    1.992516e+08
91 - Sentenças Judiciais                                                     1.832413e+08
39 - Outros Serviços de Terceiros - Pessoa Jurídica                          1.061660e+08
03 - Pensões do RPPS e do militar                                            5.790729e+07
37 - Locação de Mão-de-Obra                                                  3.447902e+07
18 - Auxílio Financeiro a Estudantes                                         3.062211e+07
46 - Auxílio-Alimentação                                                     2.565954e+07
04 - Contratação por Tempo Determinado                                       1.732586e+07
93 - Indenizações e Restituições                                             9.6

In [30]:
print('O total de gastos da Universidade de Brasília em 2020 foi de R$ '
      + str(round(sum(gastos_por_elemento),2)))

O total de gastos da Universidade de Brasília em 2020 foi de R$ 1708451320.87


In [33]:
elementos_pessoal = ['11 - Vencimentos e Vantagens Fixas - Pessoal Civil',
                     '01 - Aposentadorias do RPPS, Reserva Remunerada e Reformas dos Militares',
                     '13 - Obrigações Patronais',
                     '91 - Sentenças Judiciais',
                     '03 - Pensões do RPPS e do militar',
                     '46 - Auxílio-Alimentação',
                     '93 - Indenizações e Restituições',
                     '07 - Contribuição a Entidades Fechadas de Previdência',
                     '08 - Outros Benefícios Assistenciais do servidor e do militar',
                     '49 - Auxílio-Transporte',
                     '16 - Outras Despesas Variáveis - Pessoal Civil']

gastos_pessoal = gastos_por_elemento[elementos_pessoal]
print('O total de gastos da Universidade de Brasília com SERVIDORES em 2020 foi de R$ '
      + str(round(sum(gastos_pessoal),2))
      + ', o que representa '
      + str(round(100*sum(gastos_pessoal)/sum(gastos_por_elemento),2))
      + '% do total de gastos do ano')

O total de gastos da Universidade de Brasília com SERVIDORES em 2020 foi de R$ 1495642482.27, o que representa 87.54% do total de gastos do ano


In [34]:
gastos_2020.loc[gastos_2020.elemento_de_despesa=='37 - Locação de Mão-de-Obra']

,mes_ano,orgao_superior,orgao_ent_vinculada,unid_gestora,area_atuacao,subfuncao,programa_orcamentario,acao_orcamentaria,programa_de_governo,autor_emenda,plano_orcamentario,grupo_de_despesa,elemento_de_despesa,modalidade_de_despesa,valor_empenhado,valor_liquidado,valor_pago,restos_a_pagar_pagos
23,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,"5013 - EDUCACAO SUPERIOR - GRADUACAO, POS-GRAD...",20RK - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,00 - NAO ATRIBUIDO,0000,RO01 - REGRA DE OURO,3 - Outras Despesas Correntes,37 - Locação de Mão-de-Obra,90 - Reserva de Contingência,4924875.22,0.00,0.00,0.00
39,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,"5013 - EDUCACAO SUPERIOR - GRADUACAO, POS-GRAD...",20RK - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,00 - NAO ATRIBUIDO,0000,0000 - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,3 - Outras Despesas Correntes,37 - Locação de Mão-de-Obra,90 - Reserva de Contingência,0.00,3140642.67,2862323.29,0.00
70,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,2080 - EDUCACAO DE QUALIDADE PARA TODOS,20RK - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,00 - NAO ATRIBUIDO,0000,0000 - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,3 - Outras Despesas Correntes,37 - Locação de Mão-de-Obra,90 - Reserva de Contingência,0.00,0.00,0.00,79370.37
91,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,"5013 - EDUCACAO SUPERIOR - GRADUACAO, POS-GRAD...",8282 - REESTRUTURACAO E MODERNIZACAO DAS INSTI...,00 - NAO ATRIBUIDO,0000,0000 - REESTRUTURACAO E MODERNIZACAO DAS INSTI...,3 - Outras Despesas Correntes,37 - Locação de Mão-de-Obra,90 - Reserva de Contingência,314138.35,1680498.66,1260247.73,0.00
150,2020-08-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,2080 - EDUCACAO DE QUALIDADE PARA TODOS,8282 - REESTRUTURACAO E MODERNIZACAO DAS INSTI...,00 - NAO ATRIBUIDO,0000,0000 - REESTRUTURACAO E EXPANSAO DE INSTITUICO...,3 - Outras Despesas Correntes,37 - Locação de Mão-de-Obra,90 - Reserva de Contingência,0.00,0.00,0.00,263658.37
190,2020-03-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,"5013 - EDUCACAO SUPERIOR - GRADUACAO, POS-GRAD...",20RK - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,00 - NAO ATRIBUIDO,0000,0000 - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,3 - Outras Despesas Correntes,37 - Locação de Mão-de-Obra,90 - Reserva de Contingência,2031458.83,2053811.79,1583428.90,0.00
214,2020-03-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,2080 - EDUCACAO DE QUALIDADE PARA TODOS,20RK - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,00 - NAO ATRIBUIDO,0000,0000 - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,3 - Outras Despesas Correntes,37 - Locação de Mão-de-Obra,90 - Reserva de Contingência,0.00,0.00,0.00,1657954.60
318,2020-11-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,"5013 - EDUCACAO SUPERIOR - GRADUACAO, POS-GRAD...",20RK - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,00 - NAO ATRIBUIDO,0000,RO01 - REGRA DE OURO,3 - Outras Despesas Correntes,37 - Locação de Mão-de-Obra,90 - Reserva de Contingência,0.00,1297428.92,1294585.34,0.00
339,2020-11-01,26000 - Ministério da Educação,26271 - Fundação Universidade

In [2]:
minha_chave = API_KEY

# para conseguir acessar a API, precisamos incluir no cabeçalho da requisição a nossa chave, que definimos anteriormente. Não é necessário alterar essa variável.
chave = {'chave-api-dados':API_KEY} 

In [5]:
url = f'http://www.transparencia.gov.br/api-de-dados/despesas/por-orgao?ano=2020&orgaoSuperior=26000&orgao=26271&pagina=1'
# esta URL foi gerada a partir da consulta por órgão no swagger
   
# aqui, fazemos a consulta e obtemos o resultado para cada ano em JSON
r_orgao = requests.get(url, verify=True,headers=chave).json()

In [6]:
r_orgao

[{'ano': 2020,
  'orgao': 'Fundação Universidade de Brasília',
  'codigoOrgao': '26271',
  'orgaoSuperior': 'Ministério da Educação',
  'codigoOrgaoSuperior': '26000',
  'empenhado': '2.144.939.702,94',
  'liquidado': '1.911.492.155,27',
  'pago': '1.804.429.347,03'}]